# ODMD on pairing Hamiltonian

We test the ODMD algorithm proposed in [arXiv:2306.01858](https://arxiv.org/abs/2306.01858) on the pairing Hamiltonian. 


In [12]:
Norb = 4
Nocc = 2
gval = 0.33  

In [13]:
import numpy as np
import scipy
import itertools
from itertools import combinations
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit_aer.primitives import SamplerV2
from qiskit.circuit.library import PauliEvolutionGate, PauliGate
from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit.synthesis import SuzukiTrotter
from qiskit.visualization import plot_histogram
import seaborn as sns

In [14]:
class PairingHamiltonian:
    def __init__(self, Norb, Nocc, gval, delta_eps=1.0):
        self.Norb = Norb
        self.Nocc = Nocc
        self.delta_eps = delta_eps
        self.gval = gval
        self.basis = self.make_basis()
        self.epsilon = self.eval_epsilon()
        self.Hmat = self.eval_Hmat()

    def make_basis(self):
        self.basis = []
        for occ in combinations(range(self.Norb), self.Nocc):
            self.basis.append(occ)

        return self.basis
    
    def eval_epsilon(self):
        self.epsilon = [ 2 * i * self.delta_eps for i in range(self.Norb) ]
        return self.epsilon
    
    def eval_Hmat(self):
        dim = len(self.basis)
        self.Hmat = np.zeros((dim, dim))
        for bra_idx, bra in enumerate(self.basis):
            for ket_idx, ket in enumerate(self.basis):
                # Hamming distance
                diff = [ i for i in bra if i not in ket ]
                same = [ i for i in bra if i in ket ]
                # for SPE term
                if bra_idx == ket_idx:
                    self.Hmat[bra_idx, ket_idx] += np.sum( [self.epsilon[i] for i in same])
                    self.Hmat[bra_idx, ket_idx] += - self.gval * len(same) 
                # for pairing term
                if len(diff) == 1:
                    self.Hmat[bra_idx, ket_idx] = - self.gval

        return self.Hmat

def tuple_to_bitstring(tup, Norb, rev=True):
    bitint = 0
    for i in tup:
        bitint += 2**i
    if rev:
        bitstring = "|"+format(bitint, f'0{Norb}b')[::-1]+">"
    else:
        bitstring = "|"+format(bitint, f'0{Norb}b')+">"        
    return bitstring

def cG1(circ, c_qubit, i, j, theta):
    theta_4 = theta / 4 
    circ.cx(i,j)
    circ.ry(theta_4, i)
    circ.cx(j,i)
    circ.ry(-theta_4, i)
    circ.cx(c_qubit, i)
    circ.ry(theta_4, i)
    circ.cx(j,i)
    circ.ry(-theta_4, i)
    circ.cx(c_qubit, i)
    circ.cx(i,j)

def cG1_gate(theta):
    circ = QuantumCircuit(2)
    G(circ, 0, 1, theta)
    circ.name = "cG1"   
    circ = circ.to_gate()
    circ = circ.control(1) 
    return circ

def G(circ, i, j, theta):
    theta_2 = theta / 2 
    circ.cx(i,j)
    circ.ry(theta_2, i)
    circ.cx(j,i)
    circ.ry(-theta_2, i)
    circ.cx(j,i)
    circ.cx(i,j)  

def G_gate(theta):
    circ = QuantumCircuit(2)
    G(circ, 0, 1, theta)
    circ.name = "G"    
    return circ.to_gate()

def get_idx_ancilla_in_string(n_qubit, ancilla, Qiskit_ordering):
    idx_ancilla = None
    if ancilla != None:
        if Qiskit_ordering:
            idx_ancilla = n_qubit-1 - ancilla
        else:
            idx_ancilla = ancilla
    return idx_ancilla

def expec_Zstring(res, idx_relevant, Qiskit_ordering=True, target_qubits=[], ancilla_qubit=None):
    exp_val = exp_val_p0 = exp_val_p1 = 0.0
    n_shot = sum(res.values())
    n_qubit = len(list(res.keys())[0])
    idx_ancilla = get_idx_ancilla_in_string(n_qubit, ancilla_qubit, Qiskit_ordering)
    for bitstr, count in res.items():
        if ancilla_qubit != None and target_qubits != []:
            bitstr_target = ''.join([bitstr[k] for k in range(n_qubit) if k != idx_ancilla])
        else:
            bitstr_target = bitstr
        tmp = 1.0
        for idx in idx_relevant:
            if Qiskit_ordering:
                idx = -1 - idx
            bit = int(bitstr_target[idx])            
            tmp *= (1 - 2 * bit)
        exp_val += tmp * count
        
        if ancilla_qubit != None:
            if int(bitstr[idx_ancilla]) == 0:
                exp_val_p0 += tmp * count
            else:
                exp_val_p1 += tmp * count
    exp_val /= n_shot; exp_val_p0 /= n_shot; exp_val_p1 /= n_shot
    return exp_val, exp_val_p0, exp_val_p1


params_exact = np.array([-0.48104276, -1.03976498, -0.98963981, -1.18481738, -0.54832984])

Eshift = 0.0

def get_PairingHamiltonian(Norb, Nocc, gval, Eshift=0.0):
    Hamil = PairingHamiltonian(Norb, Nocc, gval)
    SPEs = Hamil.epsilon
    pauli_list = [ ]
    obs = [ ]
    coeffs = [ ]

    # I term
    coeff = 0.0
    op = "I" * Norb
    for i in range(Norb):
        coeff += 0.5 * ( SPEs[i] - Hamil.gval ) 
    obs += [op]
    coeff -= Eshift
    coeffs += [coeff]

    # -Zp term
    for i in range(Norb):
        op = "I" * Norb
        op = op[:i] + "Z" + op[i+1:]
        coeff = -0.5 * ( SPEs[i] - Hamil.gval )

        op = op[::-1]
        obs += [op]
        coeffs += [coeff]
    # XX+YY term
    for i in range(Hamil.Norb):
        for j in range(i+1, Hamil.Norb):
            factor = - Hamil.gval / 2
            op = "I" * Norb
            op = op[:i] + "X" + op[i+1:j] + "X" + op[j+1:]
            op = op[::-1]
            obs += [op]
            coeffs += [ factor ]
            op = "I" * Norb
            op = op[::-1]
            op = op[:i] + "Y" + op[i+1:j] + "Y" + op[j+1:]
            obs += [op]
            coeffs += [ factor ]

    return SparsePauliOp(obs, coeffs)

def ansatz(params, method="FCI"):
    qc = QuantumCircuit(Norb)
    # HF
    for i in range(Nocc):
        qc.x(i)
    # Additional gates
    if method == "FCI":
        if Norb != 4 or Nocc != 2:
            print("You are using parameters to reproduce the FCI wavefunction for Norb=4 and Nocc=2")
            print("Please make sure whether this is correct for your case!")
        qc.append(G_gate(params[0]), [1, 2])
        qc.append(G_gate(params[1]), [2, 3])
        qc.append(cG1_gate(params[2]), [2, 0, 1])
        qc.append(cG1_gate(params[3]), [3, 0, 1])
        qc.append(cG1_gate(params[4]), [3, 1, 2])        
    return qc

hamiltonian_pauli_ops = get_PairingHamiltonian(Norb, Nocc, gval, Eshift)
Hamil = PairingHamiltonian(Norb, Nocc, gval)
evals, evecs = np.linalg.eigh(Hamil.Hmat)
evals = np.linalg.eigvalsh(Hamil.Hmat)
Egs_exact = evals[0]
E_HF = Hamil.Hmat[0,0]

print("# of orbitals: ", Norb, " # of particles: ", Nocc)
print("basis:", Hamil.basis)
print([tuple_to_bitstring(tup, Norb) for tup in Hamil.basis])
#print("eps: ", Hamil.epsilon)
#print("Hmat: ", Hamil.Hmat)
print("evals: ", evals)
print("Egs_exact: ", Egs_exact, " E_HF", E_HF)
#print("gs evec", evecs[:,0])
#print("gs amp.", evecs[:,0]**2)

# of orbitals:  4  # of particles:  2
basis: [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]
['|1100>', '|1010>', '|1001>', '|0110>', '|0101>', '|0011>']
evals:  [1.18985184 3.29649666 5.34       5.34       7.42853393 9.44511758]
Egs_exact:  1.1898518351360725  E_HF 1.3399999999999999


In [15]:
def make_cU(Uprep, Ui, Ntar):
    circuit_cUi = QuantumCircuit(Ntar)
    circuit_cUi.append(Uprep, range(Ntar))
    circuit_cUi.append(Ui, range(Ntar))
    circuit_cUi = circuit_cUi.decompose().decompose()
    return circuit_cUi.to_gate().control(1)

def make_overlap_qc(Ntar, gate_cUi, gate_cUj, ancilla_qubits, target_qubits, using_statevector):
    qc_re = QuantumCircuit(1+Ntar, 1)
    qc_re.h(0)
    qc_re.append(gate_cUi, ancilla_qubits + target_qubits)
    qc_re.x(0)
    qc_re.append(gate_cUj, ancilla_qubits+target_qubits)
    qc_re.h(0)
    if not using_statevector:
        qc_re.measure(0,0)
    qc_re = qc_re.decompose()

    # Overlap, Im part 
    qc_im = QuantumCircuit(1+Ntar,1)
    qc_im.h(0)
    qc_im.append(gate_cUi, ancilla_qubits + target_qubits)
    qc_im.x(0)
    qc_im.append(gate_cUj, ancilla_qubits + target_qubits)
    qc_im.sdg(0)
    qc_im.h(0)
    if not using_statevector:
        qc_im.measure(0,0)
    qc_im = qc_im.decompose()
    return qc_re, qc_im

def measure_overlap(num_shot, Ntar, gate_cUi, gate_cUj, ancilla_qubits, target_qubits, sampler, using_statevector,
                    do_simulation=True):
    qc_re, qc_im = make_overlap_qc(Ntar, gate_cUi, gate_cUj, ancilla_qubits, target_qubits, using_statevector)
    if do_simulation:
        if using_statevector:
            results = [ Statevector.from_instruction(qc).probabilities_dict() for qc in [qc_re, qc_im]]
            prob_Re = results[0]
            prob_Im = results[1] 
        else:
            job = sampler.run([qc_re, qc_im], shots=num_shot)
            results  = job.result()
            prob_Re = results[0].data.c.get_counts()
            prob_Im = results[1].data.c.get_counts()

        p0 = np.sum( [ count for bitstr, count in prob_Re.items() if bitstr[-1] == '0' ] ) / np.sum(list(prob_Re.values()))
        p1 = np.sum( [ count for bitstr, count in prob_Re.items() if bitstr[-1] == '1' ] ) / np.sum(list(prob_Re.values()))
        ReN = p0 - p1

        p0 = np.sum( [ count for bitstr, count in prob_Im.items() if bitstr[-1] == '0' ] ) / np.sum(list(prob_Im.values()))
        p1 = np.sum( [ count for bitstr, count in prob_Im.items() if bitstr[-1] == '1' ] ) / np.sum(list(prob_Im.values()))
        ImN = p0 - p1

        U_ij = ReN + 1j * ImN
        return U_ij
    else: #only resource estimation
        print("qc_re:", dict(qc_re.count_ops()))
        print("qc_im:", dict(qc_im.count_ops()))
        return None

## ODMD

The ODMD method gives a method to estimate eigen energies of a Hamiltonian using snapshots (measurements of the overlap).


$$
s (k \Delta t) = \langle \phi_0 | e^{-i H k \Delta t} | \phi_0 \rangle
$$

$$
\begin{align}
X_{d,0:K} 
&= \begin{bmatrix}
s(0) & s(\Delta t) & \cdots & s(K \Delta t) \\
s(\Delta t) & s(2 \Delta t) & \cdots & s((K+1) \Delta t) \\
\vdots & \vdots & \ddots & \vdots \\
s((d-1) \Delta t) & s(d \Delta t) & \cdots & s((K+d-1) \Delta t)
\end{bmatrix} \\
X'=X_{d, 1:K+1}
&= \begin{bmatrix}
s(\Delta t) & s(2 \Delta t) & \cdots & s((K+1) \Delta t) \\
s(2 \Delta t) & s(3 \Delta t) & \cdots & s((K+2) \Delta t) \\
\vdots & \vdots & \ddots & \vdots \\
s(d \Delta t) & s((d+1) \Delta t) & \cdots & s((K+d) \Delta t)
\end{bmatrix} 
\end{align}
$$

If one finds $A$ such that $||X' - A X||$ is minimized, then $A$ can be used to estimate the eigenvalues of $H$.
In terms of DMD, $A$ is to be $A = X' X^+$, where $X^+$ is the pseudo-inverse of $X$.
One would then expect the eigenvalues of $A$ contain the eigenvalues of $H$.


In [16]:
# noisy simulator 
from qiskit_ibm_runtime.fake_provider import FakeQuebec
from qiskit_aer import AerSimulator

device_backend = FakeQuebec()
sim_quebec = AerSimulator.from_backend(device_backend)


In [19]:
def construct_X_and_Y(snapshots, d=8): # d is the number of delay
    N = len(snapshots)
    X = np.zeros((d, N-d), dtype=np.complex128)
    Y = np.zeros((d, N-d), dtype=np.complex128)
    for j in range(N-d):
        for i in range(d):
            idx = j + i 
            X[i,j] = snapshots[idx]
            Y[i,j] = snapshots[idx+1]
    return X, Y

def ODMD(delta_t, max_iterations, trotter_steps, num_shot, using_statevector, d=8, tol_SVD=1.e-8, verbose=False):
    ancilla_qubits=[0]
    target_qubits=list(range(1,Norb+1))
    Ntar = len(target_qubits)

    if Norb == 4 and Nocc == 2:
        Uprep = ansatz(params_exact, "FCI")
    else:
        Uprep = ansatz(dummy_params, "HF")

    cU0 = QuantumCircuit(Ntar)
    cU0.append(Uprep, range(Ntar))        
    cU0 = cU0.decompose(reps=4)
    if using_noisy_simulator:
        cU0 = transpile(cU0, backend=backend, optimization_level=2)
    cU0 = cU0.to_gate().control(1)

    snapshots = np.zeros(max_iterations, dtype=np.complex128)
    snapshots[0] = 1.0
    for it in range(1, max_iterations):
        Uj = PauliEvolutionGate(hamiltonian_pauli_ops, it*delta_t, synthesis=SuzukiTrotter(order=1,reps=trotter_steps))
        gate_cUj = make_cU(Uprep, Uj, Ntar)
        overlap = measure_overlap(num_shot, Ntar, cU0, gate_cUj, ancilla_qubits, target_qubits, sampler, using_statevector)
        snapshots[it] = overlap
    print(f"Max iteration: {max_iterations:5d} trotter_steps: {trotter_steps:5d} delta_t: {delta_t:12.8f}")

    # Observable DMD
    if verbose:
        print("snapshots of <U0|Uj>:", snapshots)
    X, Y = construct_X_and_Y(snapshots)

    # SVD of X 
    U, Sigma, Vh = np.linalg.svd(X, full_matrices=False)
    if verbose:
        print("Sigma", Sigma)

    trank = np.sum(Sigma > tol_SVD)
    Ur = U[:,:trank]
    Sigmar = np.diag(Sigma[:trank])
    Vhr = Vh[:trank,:]

    # A =Y X^+ = Y (V Sigma^-1 Udag)
    A = Y @ Vhr.T.conj() @ np.linalg.inv(Sigmar) @ Ur.T.conj()

    # Check |AX - Y| 
    if verbose:
        print("Check |AX - Y|", np.linalg.norm(A @ X - Y))

    # Eigen values of A would be exp(-iE_j dt)
    lam, v = np.linalg.eig(A)
    #print("lam", lam)

    ## argument of eigenvalues
    arg_lam = np.angle(lam)  
    #print("angle[/pi]", arg_lam / (np.pi))
    print("E?", - (arg_lam) / delta_t)

    arg_in0to2pi = arg_lam % (2 * np.pi)
    idx_E0 = np.argmax(arg_in0to2pi)
    E0 =  - arg_lam[idx_E0] / delta_t

    idx_closer = np.argmin( np.abs( Egs_exact - ( - arg_lam / delta_t )))
    E0 = - arg_lam[idx_closer] / delta_t

    print(f"E0: {E0:12.8f} abs. diff.: {np.abs(E0 - Egs_exact):9.2e}")
    print("evals", evals)
    print("")
    return E0

if __name__ == "__main__":
    
    method_num = 0 # can be 0 (statevector), 1 (shot, noiseless), 2 (shot, noisy)

    if method_num == 0:
        num_shot = 1
        using_statevector = True
        using_noisy_simulator = False
    elif method_num == 1:
        using_statevector = False 
        num_shot=10**5
        using_noisy_simulator = False
    elif method_num == 2:
        using_statevector = False 
        num_shot=10**5
        using_noisy_simulator = True

    if using_noisy_simulator:   
        device_backend = FakeQuebec()
        backend = AerSimulator.from_backend(device_backend)
        sampler = sim_quebec
    else:
        sampler = SamplerV2()

    dummy_params = [ ]

    Results = { }
    # for delta_t in [0.1, 0.01, 0.001]:
    #     for trotter_steps in [1, 10, 100]:
    #         for max_iterations in [10, 100]:
    #             E0 = ODMD(delta_t, max_iterations, trotter_steps, num_shot, using_statevector, d=8, tol_SVD=1.e-8)
    #             Results[(delta_t, trotter_steps, max_iterations)] = E0
    # print("delta_t, trotter_steps, max_iterations")
    # for tmp_key, tmp_val in Results.items():
    #     delta_t, trotter_steps, max_iterations = tmp_key
    #     tmp_val = np.abs( tmp_val - Egs_exact)
    #     print("|" + str(delta_t) + "|" + str(trotter_steps) + "|" + str(max_iterations) + "|" + str("%9.1e" % tmp_val) + "|")

    delta_t = 0.001
    trotter_steps = 100
    max_iterations = 10
    E0 = ODMD(delta_t, max_iterations, trotter_steps, num_shot, using_statevector, d=8, tol_SVD=1.e-8)
    Results[(delta_t, trotter_steps, max_iterations)] = E0

Max iteration:    10 trotter_steps:   100 delta_t:   0.00100000
E? [ 2.96543119e+03  1.18985184e+00  4.80369717e+00 -2.20201882e+03
  2.21961074e+03 -1.23653451e+03 -4.98750697e+00  1.20041852e+03]
E0:   1.18985184 abs. diff.:  5.17e-09
evals [1.18985184 3.29649666 5.34       5.34       7.42853393 9.44511758]



## Examples

- Increasing the number of iterations (over the number of dimensions of the system) does not improve the results for statevector, but it sometimes does for shot measurements.


**Norb =4, Nocc = 2, statevector, dim=6**

| delta t | trotter steps | # of iteration |abs. diff.|
|---------|---------------|-----------|----------|
|0.1|1|10|  1.5e-01|
|0.1|1|100|  3.5e-01|
|0.1|10|10|  3.3e-04|
|0.1|10|100|  1.4e-01|
|0.1|100|10|  3.4e-06|
|0.1|100|100|  9.2e-04|
|0.01|1|10|  3.3e-03|
|0.01|1|100|  3.2e-01|
|0.01|10|10|  3.3e-05|
|0.01|10|100|  6.2e-03|
|0.01|100|10|  5.0e-07|
|0.01|100|100|  2.9e-05|
|0.001|1|10|  3.3e-05|
|0.001|1|100|  6.4e-01|
|0.001|10|10|  5.1e-07|
|0.001|10|100|  6.0e-03|
|0.001|100|10|  5.2e-09|
|0.001|100|100|  4.0e-07|

**Norb =4, Nocc = 2, simulator ($10^5$ shots), dim=6**

| delta t | trotter steps | # of iteration |abs. diff.|
|---------|---------------|-----------|----------|
|0.1|1|10|  2.6e-01|
|0.1|1|100|  5.4e-01|
|0.1|10|10|  1.7e-03|
|0.1|10|100|  5.6e-02|
|0.1|100|10|  4.2e-03|
|0.1|100|100|  1.5e-03|
|0.01|1|10|  2.8e-03|
|0.01|1|100|  1.4e-01|
|0.01|10|10|  7.5e-02|
|0.01|10|100|  1.8e-04|
|0.01|100|10|  1.8e-02|
|0.01|100|100|  2.7e-03|
|0.001|1|10|  5.2e-03|
|0.001|1|100|  2.6e-03|
|0.001|10|10|  6.0e-01|
|0.001|10|100|  1.6e-02|
|0.001|100|10|  3.7e-01|
|0.001|100|100|  9.9e-03|

**Norb=6, Nocc=2, statevector, dim=15**

| delta t | trotter steps | # of iteration |abs. diff.|
|---------|---------------|-----------|----------|
|0.1|1|10|  9.1e-01|
|0.1|1|100|  3.7e-01|
|0.1|10|10|  3.7e-03|
|0.1|10|100|  2.3e-02|
|0.1|100|10|  9.7e-03|
|0.1|100|100|  3.1e-03|
|0.01|1|10|  8.3e-01|
|0.01|1|100|  4.4e-01|
|0.01|10|10|  6.1e-02|
|0.01|10|100|  6.7e-03|
|0.01|100|10|  6.2e-02|
|0.01|100|100|  2.2e-03|
|0.001|1|10|  8.2e-01|
|0.001|1|100|  9.4e-01|
|0.001|10|10|  6.2e-02|
|0.001|10|100|  8.7e-03|
|0.001|100|10|  6.3e-02|
|0.001|100|100|  1.6e-02|

**Norb=6, Nocc=2, simulator ($10^5$ shots), dim=15**

| delta t | trotter steps | # of iteration |abs. diff.|
|---------|---------------|-----------|----------|
|0.1|1|10|  2.5e-01|
|0.1|1|100|  2.2e-01|
|0.1|10|10|  1.3e-02|
|0.1|10|100|  1.4e-01|
|0.1|100|10|  1.7e-02|
|0.1|100|100|  3.8e-03|
|0.01|1|10|  2.7e-01|
|0.01|1|100|  3.7e-01|
|0.01|10|10|  2.6e-01|
|0.01|10|100|  2.4e-02|
|0.01|100|10|  2.9e-01|
|0.01|100|100|  2.3e-02|
|0.001|1|10|  1.9e-01|
|0.001|1|100|  2.6e-01|
|0.001|10|10|  1.9e-01|
|0.001|10|100|  2.2e-01|
|0.001|100|10|  5.1e-01|
|0.001|100|100|  2.6e-01|

**Norb=8, Nocc=2, statevector, dim=28**

| delta t | trotter steps | # of iteration |abs. diff.|
|---------|---------------|-----------|----------|
|0.1|1|10|  1.0e+00|
|0.1|1|100|  2.4e-01|
|0.1|10|10|  5.9e-03|
|0.1|10|100|  1.3e-01|
|0.1|100|10|  2.3e-02|
|0.1|100|100|  6.5e-03|
|0.01|1|10|  1.1e+00|
|0.01|1|100|  6.0e-01|
|0.01|10|10|  1.0e-01|
|0.01|10|100|  4.3e-02|
|0.01|100|10|  1.0e-01|
|0.01|100|100|  7.5e-03|
|0.001|1|10|  1.1e+00|
|0.001|1|100|  1.2e+00|
